In [8]:
!pip install neo4j


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from neo4j import GraphDatabase

URI = 'bolt://localhost:7687'
AUTH = (NEO4J_USERNAME, 'promact0')

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [ ]:
Node = ['Case', 'Court', 'ApplealedCase', 'CourtData', 'SummaryData', 'CitationsData']
Relationships = ['Appeal_from', 'Appeal_to', 'Is_case', 'Has_data', 'Neighbouring_data']
NodeProperties = {
    'Case':['case_name', 'case_no', 'result', 'overruled', 'overruled_by'], 
    'Court':['courst_name', 'Court_abbreviation'], 
    'ApplealedCase':['case_name', 'case_no', 'result', 'overruled', 'overruled_by'], 
    'CourtData':['type_of_case', 'petetioner', 'respondent', 'coram', 'petetioner_counsel', 'respondent_counsel', 'act', 'bench', 'dated', 'reportable', 'case_no'], 
    'SummaryData':['case_no', 'evidence', 'conclusion', 'courts_reasoning', 'precedent_analysis', 'legal_analysis',  'respondents_arguments', 'petitioners_arguments', 'issues', 'facts', 'summary'], 
    'CitationsData':['citations', 'cited', 'headnotes', 'case_referred', 'keywords','case_no']
}
Relationship_between = [
    {
        'source':'Court',
        'target':'Case',
        'relationship':'Is_case'
    },
    {
        'source':'Court',
        'target':'AppealedCase',
        'relationship':'Is_case'
    },
    {
        'source':'AppealedCase',
        'target':'Case',
        'relationship':'Appeal_from'
    },
    {
        'source':'Case',
        'target':'AppealedCase',
        'relationship':'Appeal_to'
    },
    {
        'source':'Case',
        'target':'CourtData',
        'relationship':'Has_data'
    },
    {
        'source':'Case',
        'target':'SummaryData',
        'relationship':'Has_data'
    },
    {
        'source':'Case',
        'target':'CitationsData',
        'relationship':'Has_data'
    },
    {
        'source':'CitationsData',
        'target':'CourtData',
        'relationship':'Neighbouring_data'
    },
    {
        'source':'CitationsData',
        'target':'SummaryData',
        'relationship':'Neighbouring_data'
    },
    {
        'source':'CourtData',
        'target':'CitationsData',
        'relationship':'Neighbouring_data'
    },
    {
        'source':'SummaryData',
        'target':'CitationsData',
        'relationship':'Neighbouring_data'
    },
    {
        'source':'CourtData',
        'target':'SummaryData',
        'relationship':'Neighbouring_data'
    },
    {
        'source':'SummaryData',
        'target':'CourtData',
        'relationship':'Neighbouring_data'
    },
]



In [1]:
from langchain import hub

prompt = hub.pull("hwchase17/react")

c:\Users\admin\Projects\Assignments\knowledge_graph\.venv\Lib\site-packages\langsmith\client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
c:\Users\admin\Projects\Assignments\knowledge_graph\.venv\Lib\site-packages\langsmith\client.py:5489: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [2]:
import os
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from neomodel import db, config


load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')
url = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')
neo4j_url = os.getenv('NEO4J_URI_0')
database = 'newdb'

c:\Users\admin\Projects\Assignments\knowledge_graph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
graph = Neo4jGraph(url=url, username=username, password=password, database=database)
print(graph.schema)

Node properties:
Court {court_name: STRING, court_abbreviation: STRING}
Case {case_name: STRING, result: STRING, case_no: STRING, overruled: STRING, overruled_by: STRING}
CourtData {reportable: STRING, petitioner: STRING, case_type: STRING, respondent: STRING, bench: STRING, coram: STRING, dated: STRING, case_no: STRING, petitioner_counsel: STRING, respondent_counsel: STRING, act: STRING}
SummaryData {petitioners_arguments: STRING, courts_reasoning: STRING, summary: STRING, evidence: STRING, respondents_arguments: STRING, issues: STRING, facts: STRING, case_no: STRING, conclusion: STRING, legal_analysis: STRING, precedent_analysis: STRING}
CitationsData {case_no: STRING, cited: STRING, citations: STRING, case_referred: STRING, keywords: STRING, headnotes: STRING}
Relationship properties:

The relationships:
(:Court)-[:Is_case]->(:Case)
(:Case)-[:Has_data]->(:CitationsData)
(:Case)-[:Has_data]->(:SummaryData)
(:Case)-[:Has_data]->(:CourtData)
(:CourtData)-[:Neighbouring_data]->(:Summary

In [1]:
from langchain_community.document_loaders import BSHTMLLoader


def load_html_files(file):
    try:
        loader = BSHTMLLoader(file)
        data = loader.load()
        print(f'file: {file} loaded successfully...')
        return data
    except Exception as e:
        print('error in loading file ', e)

In [15]:
from pathlib import Path
import os
from dotenv import load_dotenv
from langchain.vectorstores import Neo4jVector
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')
url = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')
neo4j_url = os.getenv('NEO4J_URI_0')
database = 'newvectordb'


def load_data(file_dir_path):
    html_files = Path(file_dir_path).glob('*.htm')

    vector_index = None
    for file in html_files:
        try:
            html_data = load_html_files(file)
            vector_index = Neo4jVector.from_documents(
                html_data,
                GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
                url=url,
                username=username,
                password=password,
                database=database,
                search_type="hybrid",
            )
            print(vector_index)
        except Exception as e:
            print(f'Error occurred while processing {file}: {e}')
    return vector_index


def get_answer(vector_index, question):
    memory = ConversationBufferMemory(
        memory_key="chat_history", return_messages=True
    )
    qa = ConversationalRetrievalChain.from_llm(
        ChatGoogleGenerativeAI(
            temperature=0,
            api_key=google_api_key,
            model='models/gemini-1.5-flash-latest'
        ),
        vector_index.as_retriever(),
        memory=memory
    )
    return qa



file_dir_path = 'C:\\Users\\admin\\Downloads\\Aritra-Docs\\Aritra-Docs'
vector_index = load_data(file_dir_path)
print('done...')


file: C:\Users\admin\Downloads\Aritra-Docs\Aritra-Docs\1197362.htm loaded successfully...
file: C:\Users\admin\Downloads\Aritra-Docs\Aritra-Docs\1197363.htm loaded successfully...
file: C:\Users\admin\Downloads\Aritra-Docs\Aritra-Docs\1197364.htm loaded successfully...
file: C:\Users\admin\Downloads\Aritra-Docs\Aritra-Docs\1197365.htm loaded successfully...
file: C:\Users\admin\Downloads\Aritra-Docs\Aritra-Docs\1197366.htm loaded successfully...
file: C:\Users\admin\Downloads\Aritra-Docs\Aritra-Docs\1197367.htm loaded successfully...
file: C:\Users\admin\Downloads\Aritra-Docs\Aritra-Docs\1197368.htm loaded successfully...
file: C:\Users\admin\Downloads\Aritra-Docs\Aritra-Docs\1197369.htm loaded successfully...
file: C:\Users\admin\Downloads\Aritra-Docs\Aritra-Docs\1197370.htm loaded successfully...
file: C:\Users\admin\Downloads\Aritra-Docs\Aritra-Docs\1197371.htm loaded successfully...
file: C:\Users\admin\Downloads\Aritra-Docs\Aritra-Docs\1197372.htm loaded successfully...
file: C:\U

In [17]:
existing_index = Neo4jVector.from_existing_index(
    GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
    url=url,
    username=username,
    password=password,
    database=database,
    search_type="hybrid",
    index_name="vector",
    keyword_index_name="text_index",
)

In [ ]:
# perform Q&A using Cypher query
question = 'Simri P.S. Case No. 64 of 2012'
answer = existing_index.similarity_search_with_score(question)
print("answer: ", answer)

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatGoogleGenerativeAI(
        temperature=0,
        api_key=google_api_key,
        model='models/gemini-1.5-flash-latest'
    ),
    chain_type="stuff",
    retriever=existing_index.as_retriever()
)

In [27]:
query = "give me all details about Purnea Sadar P.S. Case No. 393 of 2010"

ans = chain(
    {"question": query},
    return_only_outputs=True,
)
print(ans['answer'])

FINAL ANSWER: The Purnea Sadar P.S. Case No. 393 of 2010 was registered for the offence under Sections 420, 468, 471, 307 and 120B/34 of the Indian Penal Code and under Prevention of Food Adulteration Act, 1954. The First Information Report was quashed by the court.



In [11]:
query = "who was judge of the case"

chain(
    {"question": query},
    return_only_outputs=True,
)

{'answer': 'FINAL ANSWER: The judge of the case was Mr. Kishore Kumar Mandal.\n',
 'sources': 'html_data\\1197362.htm'}

In [19]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    ChatGoogleGenerativeAI(
        temperature=0,
        api_key=google_api_key,
        model='models/gemini-1.5-flash-latest'
    ), 
    existing_index.as_retriever(), 
    memory=memory
)

In [28]:
question = "tell me judge name of the case: Cr.W.J.C. No. 657 of 2016"
ans = qa({"question": question})["answer"]

print(ans)

The judge presiding over Cr.W.J.C. No. 657 of 2016 was **Mr. Birendra Kumar, J.** 



In [14]:
question = "who was judge of the case"
ans = qa({"question": question})["answer"]

print(ans)

The judge in the Simri P.S. Case No. 64 of 2012 was **Mr. Birendra Kumar**. 



In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI


load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')
url = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')
neo4j_url = os.getenv('NEO4J_URI_0')
database = 'newvectordb'

In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(url=url, username=username, password=password, database=database)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

chain.run("tell me abot the case Simri P.S. Case No. 64 of 2012 in brief")